<p><strong>Channel Attention - SE Module</strong></p>
<img src="https://images.viblo.asia/1df5dad3-4fd1-494e-bb81-f78f3a3ed1d6.png">

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Reshape, Multiply, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
def se_block(input_tensor, reduction=16):
    channel = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(channel // reduction, activation='relu')(se)
    se = Dense(channel, activation='sigmoid')(se)
    se = Reshape([1, 1, channel])(se)
    x = Multiply()([input_tensor, se])
    return x

def create_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

def create_se_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = base_model.output

    # Áp dụng SE Module cho mỗi block
    for layer in base_model.layers:
        if 'conv' in layer.name:
            x = se_block(layer.output)

    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [ ]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, batch_size=64, epochs=100):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping])

    return history, model

# Tải dữ liệu CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[-1]

170498071/170498071 [==============================] - 5s 0us/step


In [ ]:
# Tạo và huấn luyện ResNet-50
resnet50_model = create_resnet50(input_shape, num_classes)
history_resnet50, resnet50_model = train_and_evaluate(resnet50_model, X_train, y_train, X_test, y_test)


Epoch 1/100
782/782 [==============================] - 78s 54ms/step - loss: 1.9366 - accuracy: 0.3712 - val_loss: 2.9579 - val_accuracy: 0.2818
Epoch 2/100
782/782 [==============================] - 39s 50ms/step - loss: 1.7147 - accuracy: 0.4525 - val_loss: 82.6479 - val_accuracy: 0.1443
Epoch 3/100
782/782 [==============================] - 40s 51ms/step - loss: 1.7437 - accuracy: 0.4438 - val_loss: 3.1191 - val_accuracy: 0.1993
Epoch 4/100
782/782 [==============================] - 40s 51ms/step - loss: 1.6576 - accuracy: 0.4588 - val_loss: 13.3134 - val_accuracy: 0.2787
Epoch 5/100
782/782 [==============================] - 41s 53ms/step - loss: 1.4855 - accuracy: 0.5187 - val_loss: 1.5326 - val_accuracy: 0.4805
Epoch 6/100
782/782 [==============================] - 41s 53ms/step - loss: 1.2996 - accuracy: 0.5798 - val_loss: 1.3092 - val_accuracy: 0.5460
Epoch 7/100
782/782 [==============================] - 41s 53ms/step - loss: 1.3833 - accuracy: 0.5349 - val_loss: 2.7855 - val_

In [ ]:
# Tạo và huấn luyện SE-ResNet-50
se_resnet50_model = create_se_resnet50(input_shape, num_classes)
history_se_resnet50, se_resnet50_model = train_and_evaluate(se_resnet50_model, X_train, y_train, X_test, y_test)


Epoch 1/100
782/782 [==============================] - 73s 55ms/step - loss: 1.7313 - accuracy: 0.3994 - val_loss: 6.9156 - val_accuracy: 0.2736
Epoch 2/100
782/782 [==============================] - 42s 53ms/step - loss: 1.5072 - accuracy: 0.4819 - val_loss: 2.1747 - val_accuracy: 0.2902
Epoch 3/100
782/782 [==============================] - 41s 52ms/step - loss: 1.3358 - accuracy: 0.5265 - val_loss: 2.2451 - val_accuracy: 0.3090
Epoch 4/100
782/782 [==============================] - 42s 53ms/step - loss: 1.1935 - accuracy: 0.5776 - val_loss: 2.3712 - val_accuracy: 0.3827
Epoch 5/100
782/782 [==============================] - 42s 53ms/step - loss: 0.9840 - accuracy: 0.6567 - val_loss: 1.1480 - val_accuracy: 0.5943
Epoch 6/100
782/782 [==============================] - 40s 51ms/step - loss: 0.8525 - accuracy: 0.7035 - val_loss: 1.2377 - val_accuracy: 0.6045
Epoch 7/100
782/782 [==============================] - 41s 53ms/step - loss: 0.8089 - accuracy: 0.7211 - val_loss: 1.8969 - val_ac

In [ ]:
# Đánh giá mô hình
resnet50_eval = resnet50_model.evaluate(X_test, y_test)
se_resnet50_eval = se_resnet50_model.evaluate(X_test, y_test)

print("ResNet-50 Accuracy: ", resnet50_eval[1])
print("SE-ResNet-50 Accuracy: ", se_resnet50_eval[1])

313/313 [==============================] - 3s 10ms/step - loss: 0.8486 - accuracy: 0.7138
ResNet-50 Accuracy:  0.7089999914169312
SE-ResNet-50 Accuracy:  0.7138000130653381


**Spatial Attention - Deformable Convolution** <br>
<div style="text-align: center">
<img  src="https://images.viblo.asia/full/039ec952-9ceb-4c2d-b2cc-be85332d6dbc.png" >

<img src="https://images.viblo.asia/5fabf07f-80d1-44c3-8e17-b3cd9ce33ea8.png">


<img src="https://images.viblo.asia/full/70f87d1c-a3a3-450d-8aa9-179a95c9c1a3.png">

</div>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, GlobalAveragePooling2D, Dense, Reshape, Multiply, Input, BatchNormalization, ReLU, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


In [ ]:
class DeformableConv2D(Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        self.filters = filters
        self.kernel_size = kernel_size
        super(DeformableConv2D, self).__init__(**kwargs)

    def build(self, input_shape):
        self.offset_conv = Conv2D(2 * self.kernel_size * self.kernel_size,
                                  kernel_size=self.kernel_size,
                                  padding='same',
                                  use_bias=False)
        self.regular_conv = Conv2D(self.filters,
                                   kernel_size=self.kernel_size,
                                   padding='same')
        super(DeformableConv2D, self).build(input_shape)

    def call(self, inputs):
        offset = self.offset_conv(inputs)
        return self.deformable_conv2d(inputs, offset)

    def deformable_conv2d(self, x, offset):
        _, h, w, _ = x.shape
        grid_x, grid_y = tf.meshgrid(tf.range(w), tf.range(h))
        grid = tf.stack([grid_x, grid_y], axis=-1)
        grid = tf.cast(grid, tf.float32)

        offsets = tf.reshape(offset, (-1, h, w, self.kernel_size, self.kernel_size, 2))
        offsets = tf.reduce_mean(offsets, axis=[-3, -2], keepdims=True)

        grid = grid + offsets[:,:,:,0,0,:] # Extract the relevant part of offsets
        grid = tf.clip_by_value(grid, 0, h-1)
        grid = tf.round(grid)  # Convert to integer

        grid = tf.cast(grid, tf.int32)
        batch_indices = tf.range(tf.shape(x)[0])
        batch_indices = tf.reshape(batch_indices, (-1, 1, 1, 1))
        batch_indices = tf.tile(batch_indices, (1, h, w, 1))
        batch_indices = tf.cast(batch_indices, tf.int32)

        indices = tf.concat([batch_indices, grid], axis=-1) # Now both tensors have rank 4
        sampled_input = tf.gather_nd(x, indices)
        sampled_input = tf.reshape(sampled_input, (-1, h, w, x.shape[-1]))

        return self.regular_conv(sampled_input)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], input_shape[2], self.filters)

def spatial_attention(input_tensor):
    avg_pool = tf.reduce_mean(input_tensor, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(input_tensor, axis=-1, keepdims=True)
    concat = tf.concat([avg_pool, max_pool], axis=-1)
    attention = Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')(concat)
    return input_tensor * attention

In [ ]:
def create_basic_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = Conv2D(32, kernel_size=3, padding='same', activation='relu')(inputs)
    x = Conv2D(32, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_advanced_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = Conv2D(32, kernel_size=3, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # Deformable Convolution
    x = DeformableConv2D(32, kernel_size=3)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # Spatial Attention
    x = spatial_attention(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def train_and_evaluate(model, X_train, y_train, X_test, y_test, batch_size=64, epochs=100):
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping])

    return history, model


In [ ]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, batch_size=64, epochs=100):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping])

    return history, model

#Tải dữ liệu CIFAR-10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[-1]

In [ ]:
# Tạo và huấn luyện mô hình cơ bản
basic_model = create_basic_model(input_shape, num_classes)
history_basic, basic_model = train_and_evaluate(basic_model, X_train, y_train, X_test, y_test)

Epoch 1/100
782/782 [==============================] - 7s 8ms/step - loss: 1.3608 - accuracy: 0.5194 - val_loss: 1.1671 - val_accuracy: 0.5786
Epoch 2/100
782/782 [==============================] - 6s 7ms/step - loss: 0.8776 - accuracy: 0.6916 - val_loss: 1.3081 - val_accuracy: 0.5514
Epoch 3/100
782/782 [==============================] - 5s 7ms/step - loss: 0.5613 - accuracy: 0.8033 - val_loss: 1.2168 - val_accuracy: 0.6203
Epoch 4/100
782/782 [==============================] - 6s 7ms/step - loss: 0.2986 - accuracy: 0.8971 - val_loss: 1.5367 - val_accuracy: 0.6202
Epoch 5/100
782/782 [==============================] - 6s 7ms/step - loss: 0.1688 - accuracy: 0.9438 - val_loss: 3.0049 - val_accuracy: 0.5278
Epoch 6/100
782/782 [==============================] - 5s 7ms/step - loss: 0.1291 - accuracy: 0.9573 - val_loss: 2.3852 - val_accuracy: 0.5986
Epoch 7/100
782/782 [==============================] - 6s 7ms/step - loss: 0.1092 - accuracy: 0.9645 - val_loss: 2.6861 - val_accuracy: 0.5951

In [ ]:
# Tạo và huấn luyện mô hình nâng cao với Deformable Convolution và Spatial Attention
advanced_model = create_advanced_model(input_shape, num_classes)
history_advanced, advanced_model = train_and_evaluate(advanced_model, X_train, y_train, X_test, y_test)

Epoch 1/100


782/782 [==============================] - 18s 14ms/step - loss: 1.3674 - accuracy: 0.5139 - val_loss: 1.6372 - val_accuracy: 0.4622
Epoch 2/100
782/782 [==============================] - 9s 12ms/step - loss: 0.9599 - accuracy: 0.6608 - val_loss: 1.0370 - val_accuracy: 0.6333
Epoch 3/100
782/782 [==============================] - 10s 13ms/step - loss: 0.7765 - accuracy: 0.7263 - val_loss: 1.0537 - val_accuracy: 0.6267
Epoch 4/100
782/782 [==============================] - 10s 13ms/step - loss: 0.6183 - accuracy: 0.7839 - val_loss: 1.3358 - val_accuracy: 0.5883
Epoch 5/100
782/782 [==============================] - 10s 13ms/step - loss: 0.4697 - accuracy: 0.8355 - val_loss: 1.1856 - val_accuracy: 0.6328
Epoch 6/100
782/782 [==============================] - 10s 13ms/step - loss: 0.3400 - accuracy: 0.8822 - val_loss: 2.6312 - val_accuracy: 0.5071
Epoch 7/100
782/782 [==============================] - 10s 12ms/step - loss: 0.2496 - accuracy: 0.9134 - val_loss: 1.7060 - val_accuracy: 0.609

In [ ]:
# Đánh giá mô hình
basic_eval = basic_model.evaluate(X_test, y_test)
advanced_eval = advanced_model.evaluate(X_test, y_test)

print("Basic Model Accuracy: ", basic_eval[1])
print("Advanced Model Accuracy: ", advanced_eval[1])

313/313 [==============================] - 2s 5ms/step - loss: 1.0370 - accuracy: 0.6333
Basic Model Accuracy:  0.5785999894142151
Advanced Model Accuracy:  0.6333000063896179


**Channel và Spatial Attention kết hợp - CBAM**

<img src="https://images.viblo.asia/full/76cec2c1-c87d-4d4b-be85-9ece78ee81ab.png">

<img src="https://images.viblo.asia/full/976bae0c-961b-4f8b-9dd1-cfc4faf98c41.png">

<img src="https://images.viblo.asia/full/4a438a02-49f4-4e5b-bf96-afd484f22a0c.png">

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Reshape, Concatenate, Multiply, Input, BatchNormalization, ReLU, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


In [ ]:
class ChannelAttention(Layer):
    def __init__(self, filters, reduction_ratio=8, **kwargs):
        super(ChannelAttention, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        self.avg_pool = GlobalAveragePooling2D()
        self.max_pool = GlobalMaxPooling2D()
        self.dense1 = Dense(filters // reduction_ratio, activation='relu', use_bias=False)
        self.dense2 = Dense(filters, use_bias=False)

    def call(self, inputs):
        avg_pool = self.avg_pool(inputs)
        max_pool = self.max_pool(inputs)
        avg_pool = self.dense1(avg_pool)
        max_pool = self.dense1(max_pool)
        avg_pool = self.dense2(avg_pool)
        max_pool = self.dense2(max_pool)
        scale = tf.nn.sigmoid(avg_pool + max_pool)
        scale = Reshape((1, 1, self.filters))(scale)
        return inputs * scale

class SpatialAttention(Layer):
    def __init__(self, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        self.conv = Conv2D(1, kernel_size=7, strides=1, padding='same', activation='sigmoid', use_bias=False)

    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = Concatenate(axis=-1)([avg_pool, max_pool])
        scale = self.conv(concat)
        return inputs * scale

class CBAM(Layer):
    def __init__(self, filters, reduction_ratio=8, **kwargs):
        super(CBAM, self).__init__(**kwargs)
        self.channel_attention = ChannelAttention(filters, reduction_ratio)
        self.spatial_attention = SpatialAttention()

    def call(self, inputs):
        x = self.channel_attention(inputs)
        x = self.spatial_attention(x)
        return x



In [ ]:
def create_basic_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = Conv2D(32, kernel_size=3, padding='same', activation='relu')(inputs)
    x = Conv2D(32, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def create_cbam_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = Conv2D(32, kernel_size=3, padding='same', activation='relu')(inputs)
    x = CBAM(32)(x)
    x = Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    x = CBAM(64)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, batch_size=64, epochs=100):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping])

    return history, model

#Tải dữ liệu CIFAR-10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[-1]

In [ ]:
# Tạo và huấn luyện mô hình cơ bản
basic_model = create_basic_model(input_shape, num_classes)
history_basic, basic_model = train_and_evaluate(basic_model, X_train, y_train, X_test, y_test)

Epoch 1/100
782/782 [==============================] - 12s 8ms/step - loss: 1.4378 - accuracy: 0.4928 - val_loss: 1.1716 - val_accuracy: 0.5881
Epoch 2/100
782/782 [==============================] - 5s 7ms/step - loss: 0.9624 - accuracy: 0.6612 - val_loss: 1.2450 - val_accuracy: 0.5819
Epoch 3/100
782/782 [==============================] - 6s 7ms/step - loss: 0.6182 - accuracy: 0.7825 - val_loss: 1.2510 - val_accuracy: 0.6062
Epoch 4/100
782/782 [==============================] - 5s 7ms/step - loss: 0.3219 - accuracy: 0.8912 - val_loss: 1.6384 - val_accuracy: 0.5970
Epoch 5/100
782/782 [==============================] - 6s 7ms/step - loss: 0.1723 - accuracy: 0.9423 - val_loss: 1.8837 - val_accuracy: 0.6084
Epoch 6/100
782/782 [==============================] - 5s 7ms/step - loss: 0.1264 - accuracy: 0.9588 - val_loss: 2.4804 - val_accuracy: 0.5696
Epoch 7/100
782/782 [==============================] - 6s 7ms/step - loss: 0.1053 - accuracy: 0.9645 - val_loss: 3.0228 - val_accuracy: 0.583

In [ ]:
# Tạo và huấn luyện mô hình nâng cao với Deformable Convolution và Spatial Attention
cbam_model = create_cbam_model(input_shape, num_classes)
history_cbam, cbam_model = train_and_evaluate(cbam_model, X_train, y_train, X_test, y_test)

Epoch 1/100
782/782 [==============================] - 15s 15ms/step - loss: 1.9781 - accuracy: 0.2583 - val_loss: 1.8069 - val_accuracy: 0.3218
Epoch 2/100
782/782 [==============================] - 12s 15ms/step - loss: 1.6960 - accuracy: 0.3616 - val_loss: 1.5961 - val_accuracy: 0.4065
Epoch 3/100
782/782 [==============================] - 11s 14ms/step - loss: 1.5833 - accuracy: 0.4043 - val_loss: 1.5346 - val_accuracy: 0.4265
Epoch 4/100
782/782 [==============================] - 11s 14ms/step - loss: 1.5121 - accuracy: 0.4358 - val_loss: 1.4853 - val_accuracy: 0.4465
Epoch 5/100
782/782 [==============================] - 11s 14ms/step - loss: 1.4611 - accuracy: 0.4568 - val_loss: 1.4400 - val_accuracy: 0.4663
Epoch 6/100
782/782 [==============================] - 11s 15ms/step - loss: 1.4185 - accuracy: 0.4748 - val_loss: 1.4327 - val_accuracy: 0.4663
Epoch 7/100
782/782 [==============================] - 11s 14ms/step - loss: 1.3811 - accuracy: 0.4923 - val_loss: 1.3600 - val_ac

In [ ]:
# Đánh giá mô hình
basic_eval = basic_model.evaluate(X_test, y_test)
cbam_eval = cbam_model.evaluate(X_test, y_test)

print("Basic Model Accuracy: ", basic_eval[1])
print("CBAM Model Accuracy: ", cbam_eval[1])

313/313 [==============================] - 2s 5ms/step - loss: 0.9537 - accuracy: 0.6625
Basic Model Accuracy:  0.588100016117096
CBAM Model Accuracy:  0.6625000238418579
